In [92]:
# Get a news about specific topic
# To change topic (or select topic), change q = '' part
# To get specific news domain, use domains = '<news domain url>&'
# Use page=#& to get more results


# Author(maybe), title, description, url, content, publishing date, source ID 

# make dataset with CNN news article and try machine learning
# Visualized feature values


import requests
import numpy as np
import pandas as pd
import csv




# List creation

source_id = []
author = []
title = []
description = []
url = []
content = []
pub_date = []


# Read 100 news articles (20 articles per each page, 5 pages) and parse each data into corresponding lists
for i in range(1, 6):

    # News from CNN
    CNN_url = ('https://newsapi.org/v2/everything?'
           'domains=cnn.com&'
           f'page={i}&'
           'from=2020-01-27&'
           'sortBy=popularity&'
           'apiKey=ddb22f3ef2b54abe8af228db83421424')

    CNN = requests.get(CNN_url)

    print(CNN.json())
    
    for elements in CNN.json()['articles']:
        source_id.append(elements['source']['id'])
        author.append(elements['author'])
        title.append(elements['title'])
        description.append(elements['description'])
        url.append(elements['url'])
        content.append(elements['content'])
        pub_date.append(elements['publishedAt'])
        
        
        # *For CNN Only: check if the content is video and data does not related to the article
        #if (elements['content'] == 'None' or elements['content'] == "Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds."):
        #    pass
            #print(elements['description'])
        #else:
            #print (elements['content'])
        #    pass

# Test for parsed data        
#print(author)
        
    
    
# Series Creation
source_id_ser = pd.Series(source_id)
author_ser = pd.Series(author)
title_ser = pd.Series(title)
des_ser = pd.Series(description)
url_ser = pd.Series(url)
content_ser = pd.Series(content)
date_ser = pd.Series(pub_date)

News_data = pd.concat([source_id_ser, author_ser, title_ser, des_ser, url_ser, content_ser, date_ser], keys = ['source_id','author','title','description','url','content','pub_date'], axis = 1)
#print(News_data)

new_header = ['source_id', 'author', 'title', 'description', 'url', 'content', 'published_date']

#with open ('./datasets/liar_dataset/new_dataset.csv', 'w') as f:
#    writer = csv.DictWriter(f, fieldnames = new_header)

News_data.to_csv('./datasets/liar_dataset/new_dataset.csv', header = new_header)

data = pd.read_csv(f'./datasets/liar_dataset/new_dataset.csv', delimiter=',')
#print(type(data))
#print (data)

for elements in range(len(data)):
    content = data.loc[elements, 'content']
    description = data.loc[elements, 'description']

    if not pd.isnull(content):
        #print(content)
        pass

    else:
        #print(description)
        pass


{'status': 'ok', 'totalResults': 532, 'articles': [{'source': {'id': 'cnn', 'name': 'CNN'}, 'author': 'Oliver Darcy, CNN Business', 'title': 'Washington Post: Newspaper criticized for suspending reporter over Kobe Bryant tweets', 'description': "The Washington Post was criticized on Monday for placing one of its reporters on administrative leave after she generated controversy for a series of tweets posted in the wake of Kobe Bryant's death.", 'url': 'https://www.cnn.com/2020/01/27/media/washington-post-felicia-sonmez-kobe-bryant/index.html', 'urlToImage': 'https://cdn.cnn.com/cnnnext/dam/assets/200127201715-the-washington-post-file-super-tease.jpg', 'publishedAt': '2020-01-28T01:28:21Z', 'content': None}, {'source': {'id': 'cnn', 'name': 'CNN'}, 'author': 'Artemis Moshtaghian and Catherine E. Shoichet, CNN', 'title': 'Another Iranian student was detained after arriving in the US, this time in Detroit', 'description': "An Iranian PhD student who'd planned to continue his studies at Mic

PermissionError: [Errno 13] Permission denied: './datasets/liar_dataset/new_dataset.csv'

In [100]:
'''
This program is only for feature extraction from given data.

At the beginning, the program grab the news contents (if content is null, use description. If both are
null, drop that news.) from the given dataset and extract the features from the news contents.
Then, it saves extracted data and features in a file.

Features used in this program are:
1. Number of Characters
2. Number of Words
3. Number of Verbs
4. Number of Nouns
5. Number of Sentence
6. Average Number of Words per Sentence
7. Average Number of Characters in Words
8. Number of Question Marks
9. Percentage of Subjective Verbs
10. Percentage of Passive Voice
11. Percentage of Positive Words
12. Percentage of Negative Words
13. Lexical Diversity: Unique Words or Terms
14. Typographical Error Ration: Misspelled Words
15. Causation Terms

How to use: First, place positive-words.txt and negative-words.txt files in word_sentiment folder and put
            .csv dataset in liar_dataset folder in datasets folder and put the name of the .csv file
            in argument space of setup_data function. Next, use feature_extraction function with the result of
            setup_data function to extract the features and save the data in the desired file.

Date: Jan 28th, 2020
Programmed by: Sasung Kim
'''

# Importing Required modules

import numpy as np
import pandas as pd
import csv

from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from collections import Counter

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from yellowbrick.classifier import ClassificationReport
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

#########################################################################################################
# Create the lists of the words that used for feature extraction

positive_list = list()
negative_list = list()

# Gather the positive and negative word lists from the text file
with open('./word_sentiment/positive-words.txt') as p:
    for line in p:
        val = line.split()
        positive_list.append(val)

with open('./word_sentiment/negative-words.txt') as n:
    for line in n:
        val = line.split()
        negative_list.append(val)

subjective_list = list(['am ', 'are ', 'is ', 'was ', 'were ', 'be ', 'been '])
causation_list = list(['led to ', 'because ', 'cause ', 'reason ', 'explanation ', 'so '])

#########################################################################################################

# From the dataset, isolate the required data (statement and label in this program) and return as pandas Series
# If the user needs the required data as a file, uncomment def with new_file as argument and comment the other one

#def setup_data(file_name, new_file):
def setup_data(file_name):
    data = pd.read_csv(f'./datasets/liar_dataset/{file_name}', delimiter=',')

    # Check for the raw data from original dataset
    # print (list(data.columns.values))
    # print(data.tail(1))

    news_contents = []
    
    for elements in range(len(data)):
        content = data.loc[elements, 'content']
        description = data.loc[elements, 'description']
        
        if not pd.isna(content) and content is "Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.":
            news_contents.append(content)
        else:
            news_contents.append(description)
    
    # drop the null values (if both content and description are null)
    news_contents = [x for x in news_contents if x == x]
    
    # Check if the required data are correctly input
    #print(news_contents)
    
    return news_contents


#################

# Not required for right now, but may used in later for more efficient tockenization and feature extraction
# vectorizer = CountVectorizer()
# train_count = vectorizer.fit_transform(data["statement"].values)

# Check for vectorizer
# print(train_count.shape)
# print(vectorizer.vocabulary_)
# print(vectorizer.get_feature_names())

# Input: String ('str')
# Description: Tockenize and tag input with nltk universal tag
# Return: tags of each tockens - String ('tagged')
# Tockenize and tag input String with nltk universal tag and return the tags for each words in the String

def tagging_univ(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text, tagset='universal')
    return tagged

# Input: String('str')
# Description: Tockenize and tag input with nltk non-universal tag
# Return: tags of each tockens - String ('tagged')

def tagging_nuniv(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text)
    return tagged

# Input: String('str')
# Description: Count the number of characters in input
# Return: Character count - int ('count')

def count_char(str):
    no_space = str.replace(" ", "")
    count = len(no_space)
    return count

# Input: String('str')
# Description: Count the number of words in input
# Return: Word count - int ('count')

def count_word(str):
    count = len(str.split())
    return count

# Input: String ('tags'), int(verb_count)
# Description: Count the number of verbs in input
# Return: Verb count - int ('verb_count')

def count_verb(tags, verb_count):
    if (tags == 'VERB'):
        verb_count += 1
    return verb_count

# Input: String('tags'), int('noun_count')
# Description: Count the number of nouns in input
# Return: Noun count - int ('noun_count')

def count_noun(tags, noun_count):
    if (tags == 'NOUN'):
        noun_count += 1
    return noun_count

# Input: String ('str')
# Description: Count the number of sentences by counting number of period(.)
# Return: Sentence count - int ('sentence')

def count_sent(str):
    sentence = len(str.split('.'))
    return sentence

# Input: String ('states')
# Description: Count the number of characters in each word in input and average the number of characters per word
# Return: Average number of characters: float ('avg')

def count_char_per_word(states):
    word = []
    word.append(states.split())
    char_per_word = list()
    for elements in word:
        for char in elements:
            c_in_w_count = len(char)
            char_per_word.append(c_in_w_count)
    # char_per_word_list.append()
    avg = sum(char_per_word) / len(char_per_word)
    char_per_word.clear()
    return avg

# Input: String ('tagged'), list ('list'), int ('int')
# Description: Count the number of common words between input String and input list
# Return: Count of common words - int ('int')
# Count and return the number of common words between input String ('tagged') and input list

def check_common(tagged, list, int):
    for elements in list:
        if tagged in elements:
            int += 1
            return int
    return int

# Input: list ('subjective_list'), int ('verb_count'), String ('states')
# Description: Count the number of subjective words and avarage it from number of verbs in input
# Return: Average of subjective words - float ('percent_sub')

def count_sub(subjective_list, verb_count, states):
    percent_sub = 0
    for ele in subjective_list:
        sub_count = states.count(ele)
    if (verb_count > 0):
        percent_sub = sub_count / verb_count * 100
    return percent_sub

# Input: String ('word'), String ('tag'), list ('subjective_list'), int ('sent_count')
# Description: Count the number of passive voice and avarge it from number of sentence (sent_count)
# Return: Average of passive voiced sentences - float ('result')

def count_passive(word, tag, subjective_list, sent_count):
    percent_sub = 0
    counter = 0
    for ele in subjective_list:
        if (word.count(ele) > 0 and tag == "VBN"):
            counter += 1
    result = counter / sent_count * 100
    return result

# Input: String ('states')
# Description: Count the words that introduced only once in input
# Return: Count of unique words - int ('unique_count')

def count_unique(states):
    words = states.split(' ')
    c = Counter(words)
    unique = [w for w in words if c[w] == 1]
    unique_counter = len(unique)
    return unique_counter

# Input: String ('tag'), int ('typo_count')
# Description: Count the number of words with nltk universal tag is "X" (misspelled)
# Return: Count of misspelled words - int ('typo_count')

def count_typo(tag, typo_count):
    if (tag == "X"):
        typo_count += 1
    return typo_count

# Input: list ('causation_list'), String ('states')
# Description: Count the number of common words in list input and String input
# Return: Count of common words - int ('cause_count')

def count_cause(causation_list, states):
    for ele in causation_list:
        cause_count = states.count(ele)
    return cause_count

# Input: panda Series ('data'), String ('save_file_name')
# Description: First, this function creates lists for each features and extract the features using statements in dataset and
#              above functions. Next, it creates a large pandas Series that consist of news contents, labels and extracted
#              features. Finally, it save the final pandas Series in a file to make easier to examine the result (do not need
#              to rerun the program or change the code to check raw data)
# Return: Pandas Series consist of news contents, labels and extracted features count - pandas Series ('new')

def feature_extract(data, save_file_name):
    # define the news contents and labels from the dataset
    state = data

    # create lists for storing the counters
    char_count_list = list()
    word_count_list = list()
    verb_count_list = list()
    noun_count_list = list()
    sent_count_list = list()
    words_per_sent_list = list()
    char_per_word_list = list()
    quest_count_list = list()
    sub_count_list = list()
    pass_count_list = list()
    pos_count_list = list()
    neg_count_list = list()
    unique_count_list = list()
    typo_count_list = list()
    cause_count_list = list()
    word = list()

    #print(type(state))
    # loop for checking each new contents in dataset
    for states in state:

        #print(states)
        # reset the counters for each news contents
        w_in_s_count = 0
        c_in_w_count = 0
        verb_count = 0
        noun_count = 0
        pos_count = 0
        neg_count = 0
        percent_pos = 0
        percent_neg = 0
        unique_count = 0
        sent_counts = 0
        typo_count = 0

        # Tockenization and tagging with nltk universal and non-universal tag systems (tagged = universal, tagged_nu = non-universal)
        tagged = tagging_univ(states)
        tagged_nu = tagging_nuniv(states)

        # Check the tags of each news contents
        # print(tagged)
        # print(tagged_nu)

        # Extract the features and append the results in the list. Commented lines with print() functions are for testing

        # 1. Number of Characters
        char_count = count_char(states)
        char_count_list.append(char_count)

        # 2. Nubmer of Words
        word_count = count_word(states)
        word_count_list.append(word_count)

        # 3. Number of Verbs
        for tag in tagged:
            verb_count = count_verb(tag[1], verb_count)
        verb_count_list.append(verb_count)

        # 4. Number of Nouns
        for tag in tagged:
            noun_count = count_noun(tag[1], noun_count)
        noun_count_list.append(noun_count)
        # print(noun_count)

        # 5. Number of Sentence
        sent_count = count_sent(states)
        sent_count_list.append(sent_count)
        # print(sent_count)

        # 6. Average number of words per sentence
        sent = [len(l.split()) for l in re.split(r'[?!.]', states) if l.strip()]
        w_in_s_count = (sum(sent) / len(sent))
        words_per_sent_list.append(w_in_s_count)
        # print(w_in_s_count)

        # 7. Average number of characters per word
        c_in_w_count = count_char_per_word(states)
        char_per_word_list.append(c_in_w_count)
        # print(c_in_w_count)

        # 8. Number of question marks
        quest_count = states.count("?")
        quest_count_list.append(quest_count)

        # 9. Percentage of subjective verbs - am/are/is/etc
        percent_sub = count_sub(subjective_list, verb_count, states)
        sub_count_list.append(percent_sub)

        # 10. Percentage of passive voice - am/are/is && past participate
        for tag in tagged_nu:
            passive_percent = count_passive(tag[0], tag[1], subjective_list, sent_count)
        pass_count_list.append(passive_percent)

        # 11. Percentage of positive words
        for tag in tagged:
            pos_count = check_common(tag[0], positive_list, pos_count)
            percent_pos = pos_count / word_count * 100
        pos_count_list.append(percent_pos)

        # 12. Percentage of negative words
        for tag in tagged:
            neg_count = check_common(tag[0], negative_list, neg_count)
            percent_neg = neg_count / word_count * 100
        neg_count_list.append(percent_neg)

        # 13. Lexical diversity: unique words or terms
        unique_count = count_unique(states)
        unique_count_list.append(unique_count)

        # 14. Typographical error ratio: misspelled words
        for tag in tagged:
            typo_count = count_typo(tag[1], typo_count)
        typo_count_list.append(typo_count)

        # 15. Causation terms
        cause_count = count_cause(causation_list, states)
        cause_count_list.append(cause_count)

    # Convert lists into pandas Series
    statement = pd.Series(state)
    first = pd.Series(char_count_list)
    second = pd.Series(word_count_list)
    third = pd.Series(verb_count_list)
    fourth = pd.Series(noun_count_list)
    fifth = pd.Series(sent_count_list)
    sixth = pd.Series(words_per_sent_list)
    seventh = pd.Series(char_per_word_list)
    eighth = pd.Series(quest_count_list)
    nineth = pd.Series(sub_count_list)
    tenth = pd.Series(pass_count_list)
    eleventh = pd.Series(pos_count_list)
    twelveth = pd.Series(neg_count_list)
    thirteenth = pd.Series(unique_count_list)
    fourteenth = pd.Series(typo_count_list)
    fifteenth = pd.Series(cause_count_list)

    # Concatenate all feature Series, news contents and labels into one pandas Series
    new = pd.concat([statement, first, second, third, fourth, fifth, sixth, seventh, eighth, nineth, tenth,
                     eleventh, twelveth, thirteenth, fourteenth, fifteenth], axis=1)

    # Test for the final Series
    # print(new.head(3))

    # Name the header of the data
    new_header = ['Statement', '# of Characters', '# of Words', '# of Verbs', '# of Noun', '# of Sentence',
                  'Average # of Words per Sentence', 'Average # of Characters per Words', '# of Question Marks',
                  '% of Subjective Verbs', '% of Passive Voice', '% of Positive Words', '% of Negative Words',
                  '# of Unique Wrods/Terms', '# of Misspelled Words', '# of Causation Terms']

    # Save the data in a file

    #with open(save_file_name, 'w') as f:
    #    writer = csv.DictWriter(f, fieldnames=new_header)
    #    writer.writeheader()

    new.to_csv(save_file_name, header=new_header)
    # print(new)
    return new


########################################################################################################################

# Data Extraction
# From the dataset file, gather the news contents and labels only

data = setup_data('new_dataset.csv')

# Check for shape and contents of isolated data
# print(train_data.shape)
# print(test_data.shape)
# print (valid_data.shape)
# print (train_data.head(5))
# print (test_data.head(5))
# print (valid_data.head(5))

# Feature Extraction

train_data_final = feature_extract(data, './actual_news_FE.csv')


# print(valid_data_final)


['Video from KNBC shows LeBron James returning to Los Angeles after the Lakers played the Philadelphia 76ers. On January 25, James spoke with ESPN about the impact Kobe Bryant had on his career.', 'Trump spiritual adviser and White House staff member Paula White is defending her recent prayer at her church in Florida that "satanic pregnancies" be miscarried.', "CNN's Jake Tapper calls out deputy White House counsel Mike Purpura over his fourth point of defense for President Donald Trump.", 'Sen. Bernie Sanders may win the Democratic Iowa caucuses a week from today. But he lost the coveted Des Moines Register endorsement to fellow liberal Sen. Elizabeth Warren over the weekend. And, in explaining why they opted for Warren over Sanders, the Regist…', 'Netflix\'s new docuseries "Cheer," which follows Navarro College\'s cheerleading team and its bruising pursuit of yet another national championship, delves into the athletes\' lives and how they\'re shaped by sexuality, class, race and gend